# BenzDB tools - A request example

## Source code

### Module imports

In [ ]:
import ipywidgets as w
import requests
import json
import base64 
import io
from PIL import Image
import IPython as ip

### Definition of criterion classes

#### Definition of the main criterion class

In [ ]:
class Criterion:
    """ This class allows for representing criterions """
    
    def __init__ (self, key: str, description: str):
        """ initializes the criterion """
        self.__key = key
        self.__description  = description     
    
    
    def display (self):
        """ displays the widget corresponding to the criterion """
        pass
    
    
    def get_criterion (self) -> str:
        """ returns the JSON string corresponding to the criterion, an empty string if the criterion is not set """
        pass
    
    
    def get_description (self) -> str:
        """ returns the description """
        return self.__description
    
    def get_key (self) -> str:
        """ returns the key related to the criterion """
        return self.__key

#### Definition of criterion child classes

In [ ]:
class Int_Criterion (Criterion):
    """ This class allows for representing criterions based on int value """
 
    def __init__ (self, key: str, description: str, initial_value: int):
        """ initializes the criterion """
        super().__init__(key, description)
        
        self.__element =  w.IntText(value=1, layout={"width": "auto"})
        self.__condition = w.Select(description=self.get_description(), options=["not set","=","<>","<=","<",">",">="], value='not set', rows=1, layout={"width": "auto"})
                
        box = w.HBox([self.__condition, self.__element])
        display(box)
        
    
    def get_criterion (self) -> str:
        """ returns the JSON string corresponding to the criterion, an empty string if the criterion is not set """
        if self.__condition.value == "not set":
            return ""
        else:
            return '"' + self.get_key() + '": "' + self.__condition.value + " " + str(self.__element.value) + '"'

In [ ]:
class Float_Criterion (Criterion):
    """ This class allows for representing criterions based on float value """
 
    def __init__ (self, key: str, description: str, initial_value: float):
        """ initializes the criterion """
        super().__init__(key, description)
        
        self.__element =  w.IntText(layout={"width": "auto"})
        self.__condition = w.Select(description=self.get_description(), options=["not set","=","<>","<=","<",">",">="], value='not set', rows=1, layout={"width": "auto"})
                
        box = w.HBox([self.__condition, self.__element])
        display(box)
        
    
    def get_criterion (self) -> str:
        """ returns the JSON string corresponding to the criterion, an empty string if the criterion is not set """
        if self.__condition.value == "not set":
            return ""
        else:
            return '"' + self.get_key() + '": "' + self.__condition.value + " " + str(self.__element.value) + '"'

In [ ]:
class String_Criterion (Criterion):
    """ This class allows for representing criterions based on string value """
 
    def __init__ (self, key: str, description: str, initial_value: int):
        """ initializes the criterion """
        super().__init__(key, description)
        
        self.__element =  w.IntText(value=1, layout={"width": "auto"})
        self.__condition = w.Select(description=self.get_description(), options=["not set","=","<>"], value='not set', rows=1, layout={"width": "auto"})
                
        box = w.HBox([self.__condition, self.__element])
        display(box)
        
    
    def get_criterion (self) -> str:
        """ returns the JSON string corresponding to the criterion, an empty string if the criterion is not set """
        if self.__condition.value == "not set":
            return ""
        else:
            return '"' + self.get_key() + '": "' + self.__condition.value + " " + str(self.__element.value) + '"'

In [ ]:
class Query_Criterion (Criterion):
    """ This class allows for representing the desired query """
 
    def __init__ (self, key: str, description: str):
        """ initializes the criterion """
        super().__init__(key, description)
        
        self.__element =  w.Select(description=self.get_description(), options=["benzenoids","ir","ims2d1a","nics","clar_covers","properties","irregularities"], value='benzenoids', rows=1, layout={"width": "auto"})
                
        box = w.HBox([self.__element])
        display(box)
        
    
    def get_criterion (self) -> str:
        """ returns the JSON string corresponding to the criterion, an empty string if the criterion is not set """
        return str(self.__element.value)

### Definition of the form class

In [ ]:
class Form:
    
    def __init__ (self):
        """ initialise the form """
        self.__query = None         # the query
        self.__demand_type = None   # the type of demand
        self.__json_string = None   # the query as a JSON string
        self.__data = None          # the data related to the query (if the query succeeds)
        
        self.create_form()
        
        
    def create_form(self) -> None:
        """ creates the form allowing for choosing the values of desired criteria """
        self.__criteria = []

        # criteria about basic information
        self.__criteria.append (Int_Criterion(key="nbHexagons", description="# hexagons", initial_value=1))
        self.__criteria.append (Int_Criterion(key="nbCarbons", description="# carbons", initial_value=1))
        self.__criteria.append (Int_Criterion(key="nbHydrogens", description="# hydrogens", initial_value=1))

        # query
        self.__criteria.append (Query_Criterion(key="query", description="query"))    

        # validation buttons
        count_btn = w.Button (description="Count")
        count_btn.on_click (self.perform_query)

        getdata_btn = w.Button (description="Get Data")
        getdata_btn.on_click (self.perform_query)

        getquery_btn = w.Button (description="Get JSON Query")
        getquery_btn.on_click (self.perform_query)
        
        box = w.HBox([count_btn, getdata_btn,getquery_btn])
        display(box)
    
    
    def perform_query (self, btn) -> None:
        """ performs the query and sets the corresponding attributes """ 
        # we identify the type of demand
        if btn.description == "Count":
            self.__demand_type = "count"
        elif btn.description == "Get Data":
            self.__demand_type = "data"
        elif btn.description == "Get JSON Query":
            self.__demand_type = "json"
        else:
            self.__demand_type = "unknown"

        # we build the JSON string and the corres
        self.__json_string = "{\n"
        for c in self.__criteria:
            if isinstance(c,Query_Criterion):
                self.__query = c.get_criterion()
            else:
                s = c.get_criterion()
                if len(s) > 0:
                    if len(self.__json_string) > 3:
                        self.__json_string += ",\n"
                    self.__json_string += "\t" + s
        self.__json_string += "\n}"

        if self.__demand_type == "data":
            response = requests.post("https://benzenoids.lis-lab.fr/find_"+self.__query, json= json.loads(self.__json_string))
        elif self.__demand_type == "count":
            response = requests.post("https://benzenoids.lis-lab.fr/count_"+self.__query, json= json.loads(self.__json_string))
        
        if self.__demand_type in ["data","count"]:
            if response.status_code == 200:
                self.__data = response.json()

        
    def get_data (self):
        """ returns the data related to the query """
        return self.__data
    
    
    def get_query (self):
        """ returns the desired query """
        return self.__query

    
    def get_json_string (self):
        """ returns the JSON string related to the desired query """
        return self.__json_string
    

    def get_demand_type (self):
        """ returns the type of the current demand """
        return self.__demand_type

### Definition of the display classes

In [ ]:
class Display:
    """ This class allows for displaying benzenoid information """
    
    def __init__ (self, info: dict):
        """ initializes the display tool """
        self.__information = info
        
    
    def display (self) -> None:
        """ displays the information """
        print ("Benzenoid #",self.get_information("idBenzenoid"), sep="")
        print ("InChI:",self.get_information("inchi"))
        print ("Label:",self.get_information("label"))
        print ("#hexagons:",self.get_information("nbHexagons"))
        print ("#carbons:",self.get_information("nbCarbons"))
        print ("#hydrogens:",self.get_information("nbHydrogens"))
        print ("Weight:",self.get_information("weight"))
        print ("Irregularity:",self.get_information("irregularity"))
        print ("Graph file:",self.get_information("graphFile"))
        print ("Geometry:",self.get_information("geometry"))        
        
    
    def display_image (self, str64: str) -> None:
        """ displays the base-64 image defined by str64 """
        img = Image.open(io.BytesIO(base64.b64decode(str64)))
        ip.display.display(img)
        
    
    def get_information (self, key) -> dict:
        """ returns the information """
        return self.__information[key]

In [ ]:
class Display_IR (Display):
    """ This class allows for displaying benzenoid information from IR query """
    
    def display (self) -> None:
        """ displays the information """
        super().display()
        print("Final energy:",self.get_information("finalEnergy"))
        print("Frequencies:",self.get_information("frequencies")) 
        print("Intensities:",self.get_information("intensities"))
        print("AMES Format:",self.get_information("amesFormat"))

In [ ]:
class Display_IMS2D1A (Display):
    """ This class allows for displaying benzenoid information from IMS2D1A query """
    
    def display (self) -> None:
        """ displays the information """
        super().display()
        print ("VectorX:",self.get_information("vectorX"))
        print ("VectorY:",self.get_information("vectorY"))
        print ("#PointsX:",self.get_information("nbPointsX"))
        print ("#PointsY:",self.get_information("nbPointsY"))
        print ("Origin:",self.get_information("origin"))
        print ("Points:",self.get_information("points"))
        print ("Type:",self.get_information("type"))
        self.display_image(self.get_information("picture"))

In [ ]:
class Display_NICS (Display):
    """ This class allows for displaying benzenoid information from NICS query """
    
    def display (self) -> None:
        """ displays the information """
        super().display()
        print ("NICS values:", self.get_information("nics"))

In [ ]:
class Display_Clar_Covers (Display):
    """ This class allows for displaying benzenoid information from Clar Cover query """
    
    def display (self) -> None:
        """ displays the information """
        super().display()
        self.display_image(self.get_information("clarCover"))

In [ ]:
class Display_Properties (Display):
    """ This class allows for displaying benzenoid information from properties query """
    
    def display (self) -> None:
        """ displays the information """
        super().display()
        yesno = ["no","yes"]
        print ("Catacondensed:",yesno[self.get_information("catacondensed")])
        print ("Coronoid:",yesno[self.get_information("coronoid")])
        print ("Coronenoid:",yesno[self.get_information("coronenoid")])
        print ("Symmetry:",self.get_information("symmetry"))
        print ("#Kekulé structures:",self.get_information("kekule"))
        print ("HOMO:",self.get_information("homo"))
        print ("LUMO:",self.get_information("lumo"))
        print ("Dipole moment:",self.get_information("moment"))

In [ ]:
class Display_Irregularities (Display):
    """ This class allows for displaying benzenoid information from properties query """
    
    def display (self) -> None:
        """ displays the information """
        super().display()
        'solo', 'duo', 'trio', 'quartet'
        print ("# solo:",self.get_information("solo"))
        print ("# duo:",self.get_information("duo"))
        print ("# trio:",self.get_information("trio"))
        print ("# quartet:",self.get_information("quartet"))
        

## Filling the form

In [ ]:
form = Form()

## Processing the result

In [ ]:
demand_type = form.get_demand_type()

if demand_type == "data":
    # we process the result molecule per molecule
    data = form.get_data()
    query = form.get_query()
    
    for molecule in data:
        if query == "benzenoids":
            Display(molecule).display()
        elif query == "ir":
            Display_IR(molecule).display()
        elif query == "ims2d1a":
            Display_IMS2D1A(molecule).display()
        elif query == "nics":
            Display_NICS(molecule).display()
        elif query == "clar_covers":
            Display_Clar_Covers(molecule).display()
        elif query == "properties":
            Display_Properties(molecule).display()
        elif query == "irregularities":
            Display_Irregularities(molecule).display()
        print()
        
elif demand_type == "count":
    # we print the number of desired molecules
    print ("Number of molecules:",form.get_data())
    
elif demand_type == "json":
    # we print the JSON query
    print ("JSON query:")
    print (form.get_json_string())